# Debug WebSocket Connection Issue

This notebook helps debug the WebSocket connection issue with the Firefox launcher.

In [1]:
# Check what window.location is in the browser
from IPython.display import Javascript, display

display(Javascript("""
console.log('=== Browser Location Debug ===');
console.log('window.location.protocol:', window.location.protocol);
console.log('window.location.host:', window.location.host);
console.log('window.location.hostname:', window.location.hostname);
console.log('window.location.port:', window.location.port);
console.log('window.location.href:', window.location.href);
"""))

<IPython.core.display.Javascript object>

In [2]:
# Test direct connectivity to various endpoints
import requests
import subprocess
import socket

# Test JupyterHub proxy API
try:
    response = requests.get('http://127.0.0.1:8001/api/routes', timeout=5)
    print(f"JupyterHub proxy API: {response.status_code}")
    routes = response.json()
    print(f"Number of routes: {len(routes)}")
    for route, target in routes.items():
        if 'proxy/55119' in route:
            print(f"Found Firefox route: {route} -> {target}")
except Exception as e:
    print(f"Error accessing proxy API: {e}")

JupyterHub proxy API: 403
Error accessing proxy API: Expecting value: line 1 column 1 (char 0)


In [3]:
# Test direct Xpra server connectivity
def test_port(host, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except:
        return False

# Test various connectivity options
hosts_to_test = ['localhost', '127.0.0.1', '192.168.7.10']
ports_to_test = [55119, 8889]

for host in hosts_to_test:
    for port in ports_to_test:
        is_open = test_port(host, port)
        print(f"{host}:{port} - {'OPEN' if is_open else 'CLOSED'}")

localhost:55119 - CLOSED
localhost:8889 - OPEN
127.0.0.1:55119 - CLOSED
127.0.0.1:8889 - OPEN
192.168.7.10:55119 - CLOSED
192.168.7.10:8889 - OPEN


In [4]:
# Test HTTP access to proxy routes
urls_to_test = [
    'http://localhost:8889/user/bdx/proxy/55119/',
    'http://127.0.0.1:8889/user/bdx/proxy/55119/',
    'http://192.168.7.10:8889/user/bdx/proxy/55119/',
    'http://192.168.7.10:55119/',
]

for url in urls_to_test:
    try:
        response = requests.head(url, timeout=5)
        print(f"{url} - {response.status_code} {response.reason}")
    except Exception as e:
        print(f"{url} - ERROR: {e}")

http://localhost:8889/user/bdx/proxy/55119/ - 302 Found
http://127.0.0.1:8889/user/bdx/proxy/55119/ - 302 Found
http://192.168.7.10:8889/user/bdx/proxy/55119/ - 302 Found
http://192.168.7.10:55119/ - ERROR: HTTPConnectionPool(host='192.168.7.10', port=55119): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79f721ccd3d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [5]:
# Check system network configuration
import subprocess

# Check what interfaces are available
try:
    result = subprocess.run(['ip', 'addr', 'show'], capture_output=True, text=True)
    print("Network interfaces:")
    print(result.stdout)
except:
    try:
        result = subprocess.run(['ifconfig'], capture_output=True, text=True)
        print("Network interfaces:")
        print(result.stdout)
    except:
        print("Could not get network interface info")

Network interfaces:
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host noprefixroute 
       valid_lft forever preferred_lft forever
2: eno1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether 1c:69:7a:a5:f0:e4 brd ff:ff:ff:ff:ff:ff
    altname enp0s31f6
    inet6 fe80::1e69:7aff:fea5:f0e4/64 scope link 
       valid_lft forever preferred_lft forever
3: wlp0s20f3: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 64:bc:58:f7:79:48 brd ff:ff:ff:ff:ff:ff
4: eno1.7-br0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000
    link/ether be:76:91:bc:d2:c3 brd ff:ff:ff:ff:ff:ff
    inet 192.168.7.10/24 metric 100 brd 192.168.7.255 scope global dynamic e